In [1]:
import os

from utils.util_new import *
from train_test import *

In [2]:
MIND_type = 'small'
data_path = "/datasets/"

train_news_file = os.path.join(data_path, 'mind_train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'mind_train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'mind_val', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'mind_val', r'behaviors.tsv')
knowledge_graph_file = os.path.join(data_path, 'mind_kg', r'wikidata-graph.tsv')
entity_embedding_file = os.path.join(data_path, 'mind_kg', r'entity2vecd100.vec')
relation_embedding_file = os.path.join(data_path, 'mind_kg', r'relation2vecd100.vec')

mind_url, mind_train_dataset, mind_dev_dataset, _ = get_mind_data_set(MIND_type)


In [3]:
import argparse
from parse_config import ConfigParser


parser = argparse.ArgumentParser(description='KRED')

parser.add_argument('-f')


parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

In [4]:
entity2id_dict = entity_to_id(config, entities_news(config))

In [5]:
entity_embedding = []

In [6]:
entity_embedding.append(np.zeros(config['model']['entity_embedding_dim']))

In [9]:
def construct_embedding_mind(config, entity2id_dict, entity_embedding, entity2embedding_dict):
    print('constructing embedding ...')
    relation_embedding = []
    zero_array = np.zeros(config['model']['entity_embedding_dim'])
    relation_embedding.append(zero_array)
    id2entity_dict = {v: k for k, v in entity2id_dict.items()}
    with open(config['data']['entity_embedding'], 'r', encoding='utf-8') as fp_entity_embedding:
        i = 1
        for line in fp_entity_embedding:
            if i in id2entity_dict:
              linesplit = line.strip().split('\t')
              linesplit = [float(i) for i in linesplit]
              entity2embedding_dict[id2entity_dict[i]] = len(entity_embedding)
              entity_embedding.append(linesplit)
            i += 1
    with open(config['data']['relation_embedding'], 'r', encoding='utf-8') as fp_relation_embedding:
        for line in fp_relation_embedding:
            linesplit = line.strip().split('\t')
            linesplit = [float(i) for i in linesplit]
            relation_embedding.append(linesplit)
    return entity2embedding_dict, entity_embedding, relation_embedding

In [11]:
entity2embedding_dict = {}
entity2embedding_dict, entity_embedding, relation_embedding = construct_embedding_mind(config, entity2id_dict, entity_embedding, entity2embedding_dict)


constructing embedding ...


In [12]:
entity_adj, relation_adj = construct_adj_mind(config, entity2id_dict, entity2embedding_dict)

TypeError: construct_adj_mind() takes 1 positional argument but 3 were given